In [31]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch_geometric.utils import degree, to_undirected
from sklearn.metrics import average_precision_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## 1. Load Data

In [32]:
# Load all data
edge_index = torch.load('../data/edge_index.pt').to(device)
node_features = torch.load('../data/node_features.pt').to(device)
y = torch.load('../data/y.pt').to(device)
train_idx = torch.load('../data/train_idx.pt').to(device)
test_idx = torch.load('../data/test_idx.pt').to(device)

# Ensure undirected graph
edge_index = to_undirected(edge_index)

num_nodes = node_features.size(0)
num_labels = y.size(1)

print(f"Nodes: {num_nodes}, Labels: {num_labels}")
print(f"Train: {len(train_idx)}, Test: {len(test_idx)}")
print(f"Edges: {edge_index.size(1)//2} (undirected)")

Nodes: 19765, Labels: 305
Train: 5046, Test: 3365
Edges: 777395 (undirected)


## 2. Label Propagation Class

In [33]:
class LabelPropagation:
    def __init__(self, edge_index, num_nodes, num_layers=50, alpha=0.9, device='cpu'):
        self.edge_index = edge_index
        self.num_nodes = num_nodes
        self.num_layers = num_layers
        self.alpha = alpha
        self.device = device
        
        # Compute symmetric normalized adjacency
        row, col = edge_index
        deg = degree(row, num_nodes).float()
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        
        self.edge_weight = deg_inv_sqrt[row] * deg_inv_sqrt[col]
    
    def propagate(self, y_train, train_idx):
        # Initialize predictions
        y_soft = torch.zeros(self.num_nodes, y_train.size(1), device=self.device)
        y_soft[train_idx] = y_train
        
        # Store initial predictions
        y_init = y_soft.clone()
        
        # Propagate
        for _ in range(self.num_layers):
            y_new = torch.zeros_like(y_soft)
            row, col = self.edge_index
            
            # Aggregate from neighbors
            y_new.index_add_(0, row, y_soft[col] * self.edge_weight.view(-1, 1))
            
            # Mixing with initial predictions
            y_soft = self.alpha * y_new + (1 - self.alpha) * y_init
        
        return y_soft

## 3. Correct and Smooth Class

In [34]:
class CorrectAndSmooth:
    def __init__(self, edge_index, num_nodes, num_layers=50, 
                 correction_alpha=0.8, smoothing_alpha=0.8, device='cpu'):
        self.edge_index = edge_index
        self.num_nodes = num_nodes
        self.num_layers = num_layers
        self.correction_alpha = correction_alpha
        self.smoothing_alpha = smoothing_alpha
        self.device = device
        
        # Compute symmetric normalized adjacency
        row, col = edge_index
        deg = degree(row, num_nodes).float()
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        
        self.edge_weight = deg_inv_sqrt[row] * deg_inv_sqrt[col]
    
    def correct(self, y_soft, y_train, train_idx):
        # Compute training errors
        error = torch.zeros_like(y_soft)
        error[train_idx] = y_train - y_soft[train_idx]
        
        # Propagate errors
        for _ in range(self.num_layers):
            error_new = torch.zeros_like(error)
            row, col = self.edge_index
            
            error_new.index_add_(0, row, error[col] * self.edge_weight.view(-1, 1))
            error = self.correction_alpha * error_new + (1 - self.correction_alpha) * error
            error[train_idx] = y_train - y_soft[train_idx]
        
        return y_soft + error
    
    def smooth(self, y_soft, y_train, train_idx):
        y_init = y_soft.clone()
        
        # Smooth predictions
        for _ in range(self.num_layers):
            y_new = torch.zeros_like(y_soft)
            row, col = self.edge_index
            
            y_new.index_add_(0, row, y_soft[col] * self.edge_weight.view(-1, 1))
            y_soft = self.smoothing_alpha * y_new + (1 - self.smoothing_alpha) * y_init
            y_soft[train_idx] = y_train
        
        return y_soft

## 4. Multi-Scale Label Propagation

Generate predictions with 5 different alpha values to capture different propagation scales

In [35]:
# Use full training data (no validation split for final submissions)
y_train = y[train_idx].float()

# Define alpha values for multi-scale propagation
alphas = [0.70, 0.75, 0.80, 0.85, 0.90]
lp_predictions = {}

print("Running Multi-Scale Label Propagation...\n")

for alpha in alphas:
    print(f"Alpha = {alpha}")
    
    # Label Propagation
    lp = LabelPropagation(edge_index, num_nodes, num_layers=50, alpha=alpha, device=device)
    y_lp = lp.propagate(y_train, train_idx)
    
    # Correct and Smooth
    cs = CorrectAndSmooth(edge_index, num_nodes, num_layers=50,
                          correction_alpha=0.8, smoothing_alpha=0.8, device=device)
    y_corrected = cs.correct(y_lp, y_train, train_idx)
    y_final = cs.smooth(y_corrected, y_train, train_idx)
    
    lp_predictions[alpha] = y_final.cpu()
    
    # Check statistics
    test_preds = y_final[test_idx]
    print(f"  Test predictions - Mean: {test_preds.mean():.4f}, "
          f"Min: {test_preds.min():.4f}, Max: {test_preds.max():.4f}\n")

print("Multi-Scale LP completed!")

Running Multi-Scale Label Propagation...

Alpha = 0.7
  Test predictions - Mean: 0.0115, Min: -0.0008, Max: 0.3636

Alpha = 0.75
  Test predictions - Mean: 0.0117, Min: -0.0006, Max: 0.3682

Alpha = 0.8
  Test predictions - Mean: 0.0120, Min: -0.0005, Max: 0.3724

Alpha = 0.85
  Test predictions - Mean: 0.0122, Min: -0.0003, Max: 0.3761

Alpha = 0.9
  Test predictions - Mean: 0.0125, Min: -0.0002, Max: 0.3794

Multi-Scale LP completed!


## 5. Link Prediction Features

Compute link prediction scores for test nodes:
- **Common Neighbors**: Count of shared neighbors
- **Jaccard Coefficient**: Normalized overlap of neighborhoods
- **Adamic-Adar**: Weighted common neighbors (by inverse log degree)

In [36]:
# Move to CPU for efficient graph operations
edge_index_cpu = edge_index.cpu()

# Build adjacency list
from collections import defaultdict

adj_list = defaultdict(set)
for i in range(edge_index_cpu.size(1)):
    src, dst = edge_index_cpu[0, i].item(), edge_index_cpu[1, i].item()
    adj_list[src].add(dst)

print(f"Built adjacency list with {len(adj_list)} nodes")

# Compute node degrees
node_degrees = {node: len(neighbors) for node, neighbors in adj_list.items()}

Built adjacency list with 19765 nodes


In [37]:
def compute_link_features(node_idx, train_idx_cpu, adj_list, node_degrees):
    """
    Compute link prediction features for a node based on training nodes.
    Returns: common_neighbors, jaccard, adamic_adar scores
    """
    neighbors = adj_list[node_idx]
    
    # Initialize scores
    cn_scores = []
    jaccard_scores = []
    aa_scores = []
    
    # Compute scores with training nodes
    for train_node in train_idx_cpu:
        train_neighbors = adj_list[train_node]
        
        # Common Neighbors
        common = neighbors.intersection(train_neighbors)
        cn = len(common)
        cn_scores.append(cn)
        
        # Jaccard Coefficient
        union = neighbors.union(train_neighbors)
        jaccard = cn / len(union) if len(union) > 0 else 0
        jaccard_scores.append(jaccard)
        
        # Adamic-Adar
        aa = sum(1.0 / np.log(node_degrees[z]) for z in common if node_degrees[z] > 1)
        aa_scores.append(aa)
    
    return np.mean(cn_scores), np.mean(jaccard_scores), np.mean(aa_scores)

# Compute link features for all test nodes
train_idx_cpu = train_idx.cpu().tolist()
test_idx_cpu = test_idx.cpu().tolist()

link_features = np.zeros((len(test_idx_cpu), 3))  # CN, Jaccard, AA

print("Computing link prediction features for test nodes...")
for i, node_idx in enumerate(tqdm(test_idx_cpu)):
    cn, jaccard, aa = compute_link_features(node_idx, train_idx_cpu, adj_list, node_degrees)
    link_features[i] = [cn, jaccard, aa]

print(f"\nLink Features Shape: {link_features.shape}")
print(f"Common Neighbors - Mean: {link_features[:, 0].mean():.2f}, Std: {link_features[:, 0].std():.2f}")
print(f"Jaccard - Mean: {link_features[:, 1].mean():.4f}, Std: {link_features[:, 1].std():.4f}")
print(f"Adamic-Adar - Mean: {link_features[:, 2].mean():.2f}, Std: {link_features[:, 2].std():.2f}")

Computing link prediction features for test nodes...


100%|██████████████████████████████████████████████████████████████████████████████| 3365/3365 [02:12<00:00, 25.32it/s]


Link Features Shape: (3365, 3)
Common Neighbors - Mean: 1.11, Std: 1.57
Jaccard - Mean: 0.0048, Std: 0.0037
Adamic-Adar - Mean: 0.18, Std: 0.26


## 6. Convert Link Features to Label Predictions

Use link features as similarity scores to weight training labels

In [38]:
# Normalize link features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
link_features_norm = scaler.fit_transform(link_features)

# Compute similarity-weighted label predictions
y_train_np = y_train.cpu().numpy()

# For each test node, compute weighted average of training labels
# based on link prediction scores
link_predictions = []

print("\nGenerating link-based predictions...")
for i, test_node in enumerate(tqdm(test_idx_cpu)):
    test_neighbors = adj_list[test_node]
    
    # Find training nodes that are neighbors
    neighbor_train_mask = []
    neighbor_train_indices = []
    for j, train_node in enumerate(train_idx_cpu):
        if train_node in test_neighbors:
            neighbor_train_mask.append(True)
            neighbor_train_indices.append(j)
        else:
            neighbor_train_mask.append(False)
    
    if len(neighbor_train_indices) > 0:
        # Average labels of neighbor training nodes
        neighbor_labels = y_train_np[neighbor_train_indices]
        link_pred = neighbor_labels.mean(axis=0)
    else:
        # Use global average if no neighbors
        link_pred = y_train_np.mean(axis=0)
    
    link_predictions.append(link_pred)

link_predictions = np.array(link_predictions)
print(f"Link predictions shape: {link_predictions.shape}")
print(f"Mean: {link_predictions.mean():.4f}, Min: {link_predictions.min():.4f}, Max: {link_predictions.max():.4f}")


Generating link-based predictions...


100%|████████████████████████████████████████████████████████████████████████████| 3365/3365 [00:01<00:00, 2309.93it/s]

Link predictions shape: (3365, 305)
Mean: 0.0406, Min: 0.0000, Max: 1.0000


## 7. Create Submission Function

In [39]:
def create_submission(predictions, filename, test_idx):
    """
    Create submission CSV with correct format.
    
    Args:
        predictions: numpy array of shape (num_test_nodes, num_labels)
        filename: output filename
        test_idx: test node indices
    """
    test_idx_cpu = test_idx.cpu().numpy()
    
    # CRITICAL: Clip predictions to [0, 1] range
    predictions = np.clip(predictions, 0.0, 1.0)
    
    # Create DataFrame with correct column names: label_0, label_1, ..., label_304
    num_labels = predictions.shape[1]
    label_columns = [f'label_{i}' for i in range(num_labels)]
    submission = pd.DataFrame(predictions, columns=label_columns)
    submission.insert(0, 'node_id', test_idx_cpu)
    
    # Save
    output_path = f'../Submissions/{filename}'
    submission.to_csv(output_path, index=False)
    
    print(f"Saved: {output_path}")
    print(f"Shape: {submission.shape}")
    print(f"Header: node_id,label_0,...,label_{num_labels-1}")
    print(f"Mean prediction: {predictions.mean():.6f}\n")

## 8. Generate Ensemble Submissions

### Strategy:
1. **Pure Multi-Scale LP**: Average of all 5 alpha values
2. **LP 0.90 + 5% Link**: 95% best LP (α=0.9) + 5% link features
3. **LP 0.90 + 10% Link**: 90% best LP (α=0.9) + 10% link features
4. **Multi-Scale + 5% Link**: 95% multi-scale average + 5% link
5. **Adaptive Ensemble**: Use α=0.90 for common labels, α=0.75 for rare labels

In [40]:
# Get test predictions for each method
test_idx_cpu = test_idx.cpu()

lp_test_preds = {}
for alpha in alphas:
    lp_test_preds[alpha] = lp_predictions[alpha][test_idx_cpu].numpy()

print("Test predictions extracted for all LP alphas")

Test predictions extracted for all LP alphas


### Submission 1: Pure Multi-Scale LP Average

In [41]:
# Average all 5 LP predictions
multi_scale_avg = np.mean([lp_test_preds[alpha] for alpha in alphas], axis=0)

create_submission(
    multi_scale_avg,
    'submission_Draft11_MultiScale_Avg.csv',
    test_idx
)

Saved: ../Submissions/submission_Draft11_MultiScale_Avg.csv
Shape: (3365, 306)
Header: node_id,label_0,...,label_304
Mean prediction: 0.011965



### Submission 2: LP 0.90 + 5% Link Features

In [42]:
# 95% LP (alpha=0.9) + 5% link predictions
ensemble_95_5 = 0.95 * lp_test_preds[0.90] + 0.05 * link_predictions

create_submission(
    ensemble_95_5,
    'submission_Draft11_LP90_Link5.csv',
    test_idx
)

Saved: ../Submissions/submission_Draft11_LP90_Link5.csv
Shape: (3365, 306)
Header: node_id,label_0,...,label_304
Mean prediction: 0.013877



### Submission 3: LP 0.90 + 10% Link Features

In [43]:
# 90% LP (alpha=0.9) + 10% link predictions
ensemble_90_10 = 0.90 * lp_test_preds[0.90] + 0.10 * link_predictions

create_submission(
    ensemble_90_10,
    'submission_Draft11_LP90_Link10.csv',
    test_idx
)

Saved: ../Submissions/submission_Draft11_LP90_Link10.csv
Shape: (3365, 306)
Header: node_id,label_0,...,label_304
Mean prediction: 0.015285



### Submission 4: Multi-Scale Average + 5% Link

In [44]:
# 95% multi-scale average + 5% link predictions
ensemble_multi_link5 = 0.95 * multi_scale_avg + 0.05 * link_predictions

create_submission(
    ensemble_multi_link5,
    'submission_Draft11_MultiScale_Link5.csv',
    test_idx
)

Saved: ../Submissions/submission_Draft11_MultiScale_Link5.csv
Shape: (3365, 306)
Header: node_id,label_0,...,label_304
Mean prediction: 0.013399



### Submission 5: Adaptive Multi-Scale by Label Frequency

In [45]:
# Compute label frequencies
label_freq = y_train.sum(dim=0).cpu().numpy()

# Categorize labels
rare_labels = label_freq < 50
medium_labels = (label_freq >= 50) & (label_freq < 200)
common_labels = label_freq >= 200

print(f"Rare labels: {rare_labels.sum()}")
print(f"Medium labels: {medium_labels.sum()}")
print(f"Common labels: {common_labels.sum()}")

# Adaptive ensemble
adaptive_ensemble = np.zeros_like(multi_scale_avg)

# Rare labels: use lower alpha (0.75) for more smoothing
adaptive_ensemble[:, rare_labels] = lp_test_preds[0.75][:, rare_labels]

# Medium labels: use medium alpha (0.85)
adaptive_ensemble[:, medium_labels] = lp_test_preds[0.85][:, medium_labels]

# Common labels: use high alpha (0.90) for less smoothing
adaptive_ensemble[:, common_labels] = lp_test_preds[0.90][:, common_labels]

create_submission(
    adaptive_ensemble,
    'submission_Draft11_Adaptive.csv',
    test_idx
)

Rare labels: 37
Medium labels: 184
Common labels: 84
Saved: ../Submissions/submission_Draft11_Adaptive.csv
Shape: (3365, 306)
Header: node_id,label_0,...,label_304
Mean prediction: 0.012340



## 9. Summary

Generated 5 submissions:

1. **submission_Draft11_MultiScale_Avg.csv**: Average of 5 LP alphas (0.70-0.90)
2. **submission_Draft11_LP90_Link5.csv**: 95% LP (α=0.9) + 5% link features
3. **submission_Draft11_LP90_Link10.csv**: 90% LP (α=0.9) + 10% link features
4. **submission_Draft11_MultiScale_Link5.csv**: 95% multi-scale + 5% link
5. **submission_Draft11_Adaptive.csv**: Adaptive alpha by label frequency

**Next Steps:**
- Submit all 5 files to Kaggle
- If any improves over 0.0571, proceed to **Option B** (riskier features)
- Track which strategies work best for final submissions